In [ ]:
import numpy
import pygetm

%matplotlib widget
import matplotlib.pyplot

In [ ]:
halo = 2
Lx, Ly = 100., 100.

domain = pygetm.domain.Domain.create_cartesian(numpy.linspace(-Lx/2, Lx/2, 100), numpy.linspace(-Ly/2, Ly/2, 101), 1, H=1., lat=0., mask=1)
domain.initialize(runtype=1)

In [ ]:
period = 600
omega = 2 * numpy.pi / period
cfl = 1.
umax = omega * Lx / 2
dt_cfl = cfl * min(Lx / (domain.T.H.shape[1] + 1), Ly / (domain.T.H.shape[0] + 1)) / umax
no_of_revolutions = 5
Nmax = no_of_revolutions * round(2 * numpy.pi / omega / dt_cfl)
tmax = no_of_revolutions * 2 * numpy.pi / omega
timestep = tmax / Nmax

In [ ]:
u = domain.U.array(fill=0.)
v = domain.V.array(fill=0.)
var = domain.T.array(fill=1.)
u[:, :] = -omega * domain.U.y
v[:, :] = omega * domain.V.x
u[(2 * domain.U.x / Lx)**2 + (2 * domain.U.y / Ly)**2 > 1] = 0.
v[(2 * domain.V.x / Lx)**2 + (2 * domain.V.y / Ly)**2 > 1] = 0.
var[int(0.2 * domain.shape[1]):int(0.4 * domain.shape[1]), int(0.2 * domain.shape[2]):int(0.4 * domain.shape[2])] = 5.

In [ ]:
fig, ax = matplotlib.pyplot.subplots()
u_destag, v_destag = u.interp(domain.T), v.interp(domain.T)
pc = ax.quiver(u_destag[::5,::5], v_destag[::5,::5])

In [ ]:
nsample = 4
fig, ax = matplotlib.pyplot.subplots()
pc = ax.pcolormesh(var)
cb = fig.colorbar(pc)
title = ax.set_title('time step: 0 of %i' % Nmax)

In [ ]:
totals = [var[...].sum()]
adv = pygetm.Advection(domain.T, scheme=1)
for i in range(1, Nmax):
    adv.calculate(u, v, timestep, var)
    if i % nsample == 0:
        pc.set_array(var[...].ravel())
        title.set_text('time step: %i of %i' % (i, Nmax))
        fig.canvas.draw()
    totals.append(var[...].sum())

In [ ]:
fig, ax = matplotlib.pyplot.subplots()
ax.plot(totals / totals[0] - 1.)
ax.grid()
ax.set_xlabel('time step')
ax.set_ylabel('change in total tracer')
fig.show()